In [1]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import json

/opt/conda/envs/travelplanner/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
true_data = pd.read_csv("data/test_csv_rlhf_gpt_4o.csv")
# false_data = pd.read_csv('test_csv_rlhf_qwen1.5b.csv')

# combined_df = pd.concat([true_data, false_data], ignore_index=True)

combined_df = pd.read_csv("data/test_csv_rlhf_10_to_30_gpt_4o.csv")

combined_df = combined_df.rename(columns= {'completion': 'gpt'})
combined_df = combined_df[combined_df['label'] == True]

In [6]:
true_data.head()

,prompt,completion,label
0,"[{""content"": ""Collect information for a query ...","[{""content"": ""I need to gather information abo...",True
1,"[{""content"": ""Collect information for a query ...","[{""content"": ""FlightSearch[Washington, Myrtle ...",True
2,"[{""content"": ""Collect information for a query ...","[{""content"": ""I have found two flight options ...",True
3,"[{""content"": ""Collect information for a query ...","[{""content"": ""AccommodationSearch[Myrtle Beach...",True
4,"[{""content"": ""Collect information for a query ...","[{""content"": ""I need to analyze the accommodat...",True


In [8]:
len(combined_df)

552

In [ ]:
# Set the device (cuda for GPU, cpu for CPU)
device = "cuda"  # or "cpu" if you don't have a GPU

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)


def swap_content_role(s):
    s = s.replace('\n', '\\n').replace('\t', '\\t').replace('\\"', '\\\\"')

    parsed_data = json.loads(s)
    formatted_output = [
        {'role': entry['role'], 'content': entry['content']} for entry in parsed_data
    ]
    return formatted_output


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

#prompt_str = combined_df['prompt'][0]

def get_hf_output(prompt_str):
        
    try:
        messages = swap_content_role(prompt_str)
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512
        )

        response = tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('assistant')[-1]

        return [{"content": response, "role": "assistant"}]
    
    except:
        return [{"content": " ", "role": "assistant"}]


#print(get_hf_output(prompt_str))

combined_df['qwen'] = combined_df['prompt'].apply(get_hf_output)

/opt/conda/envs/travelplanner/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


> /tmp/ipykernel_63768/3906774161.py(48)get_hf_output()
     46         pdb.set_trace()
     47 
---> 48         return [{"content": response, "role": "assistant"}]
     49 
     50     except:

"\nTo find flights from Minneapolis to Seattle, I'll first use the FlightSearch function.\nAction 1: FlightSearch[Minneapolis, Seattle, 2022-03-29]"
[{'content': "\nTo find flights from Minneapolis to Seattle, I'll first use the FlightSearch function.\nAction 1: FlightSearch[Minneapolis, Seattle, 2022-03-29]", 'role': 'assistant'}]
> /tmp/ipykernel_63768/3906774161.py(48)get_hf_output()
     46         pdb.set_trace()
     47 
---> 48         return [{"content": response, "role": "assistant"}]
     49 
     50     except:

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/ipykernel_63768/3906774161.py(48)get_hf_output()
     46         pdb.set_trace()
     47 
---> 48         return [{"content": response, "role": "assistant"}]
     49 
     50     except:

> /tmp/ipykernel_63

In [ ]:
combined_df.to_pickle("combined_df_10_30_samples.pkl")

In [3]:

#restarted


combined_df = pd.read_pickle("combined_df.pkl")
def content_role(row):
    print()
    prompt = json.loads(row['prompt'].replace('\n', '\\n').replace('\t', '\\t').replace('\\"', '\\\\"'))
    gpt = json.loads(row['gpt'].replace('\n', '\\n').replace('\t', '\\t').replace('\\"', '\\\\"'))
    qwen = row['qwen']
    
    return {'prompt': prompt[0]['content'], 'chosen': prompt + gpt, 'rejected': prompt + qwen}

    # return prompt[0]['content'], prompt + gpt, prompt + qwen

train_list = []
for _, row in combined_df.iterrows():
    train_list.append(content_role(row))

train_df = pd.DataFrame(train_list)



In [4]:

# train_dpo.py
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

model_path = "Qwen/Qwen2-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


# Assuming your data is in a CSV file
train_dataset = Dataset.from_pandas(train_df)




In [5]:


model_output_dir = model_path + str("_DPO")

training_args = DPOConfig(
    output_dir=model_output_dir,
    logging_steps=10,
    per_device_train_batch_size=1,  # Reduce this value
    gradient_accumulation_steps=16,   # Increase this to maintain effective batch size
    fp16=True,
)


from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

#trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=train_dataset,
)
trainer.train()

Tokenizing train dataset: 100%|██████████| 10/10 [00:00<00:00, 175.52 examples/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 212.00 MiB. GPU 0 has a total capacty of 22.19 GiB of which 129.50 MiB is free. Including non-PyTorch memory, this process has 22.06 GiB memory in use. Of the allocated memory 21.47 GiB is allocated by PyTorch, and 291.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [103]:
import torch
torch.cuda.empty_cache()

In [1]:
!nvidia-smi

Mon Dec  9 02:38:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   25C    P8              16W / 300W |      2MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
